🧠 Ejercicio 5: Rendimiento por club en una temporada
Objetivo: Calcular partidos jugados, ganados, perdidos y empatados por club en la temporada 2022/2023.

Archivos requeridos:

games.csv

clubs.csv

Instrucciones:

Filtra los juegos con season == '2022/2023'.

Por cada club:

Cuenta partidos jugados como local o visitante.

Cuenta ganados, perdidos y empatados.

Une con clubs para obtener el nombre del club.

Muestra el ranking por mayor cantidad de partidos ganados.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

games = spark.read.option('header','True').csv('../../data/practica/games.csv')
clubs = spark.read.option('header','True').csv('../../data/practica/clubs.csv')

In [2]:
games.printSchema()
clubs.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- season: string (nullable = true)
 |-- round: string (nullable = true)
 |-- date: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_club_id: string (nullable = true)
 |-- home_club_goals: string (nullable = true)
 |-- away_club_goals: string (nullable = true)
 |-- home_club_position: string (nullable = true)
 |-- away_club_position: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)

root
 |-- club_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: string (nullable = true)
 |-- squad_size: string (nullable = true)
 |-- average_age: string (nullable = true)
 |-- foreigners_number: string (nullable = tru

In [35]:
from pyspark.sql.functions import col,desc,when,count,sum
g = games.alias('g')
c = clubs.alias('c')

join_p1 = g.join(c, c.club_id == g.home_club_id)

df_calculo = join_p1.withColumn('resultado', 
                                when(g.home_club_goals > g.away_club_goals , 'WIN').
                                when(g.home_club_goals < g.away_club_goals , 'LOSE').
                                otherwise('DRAW')
                                )

In [20]:
df_calculo.printSchema()


root
 |-- game_id: string (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- season: string (nullable = true)
 |-- round: string (nullable = true)
 |-- date: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_club_id: string (nullable = true)
 |-- home_club_goals: string (nullable = true)
 |-- away_club_goals: string (nullable = true)
 |-- home_club_position: string (nullable = true)
 |-- away_club_position: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: string (nullable = true)
 |-- squad_size: string (nullable = true)
 |-- average_age: string (nullable = true)
 |-- foreigners_number: string (nullable = true)
 |-

In [36]:
#groupBy(clubs)
#count(game_id)
#count(game_id).filter(resultado == 'WIN')
#count(game_id).filter(resultado == 'LOSE')
#count(game_id).filter(resultado == 'DRAW')
df_resultado = df_calculo.groupBy("name").agg(
    count("game_id").alias("TOTAL_JUEGOS"),
    sum(when(col("resultado") == "WIN", 1).otherwise(0)).alias("GANADOS"),
    sum(when(col("resultado") == "DRAW", 1).otherwise(0)).alias("EMPATADOS"),
    sum(when(col("resultado") == "LOSE", 1).otherwise(0)).alias("PERDIDOS")
)


In [38]:
df_resultado.orderBy(desc('TOTAL_JUEGOS')).show(5)

+--------------+------------+-------+---------+--------+
|          name|TOTAL_JUEGOS|GANADOS|EMPATADOS|PERDIDOS|
+--------------+------------+-------+---------+--------+
|  fc-barcelona|         214|    167|       27|      20|
|    fc-sevilla|         207|    144|       35|      28|
|   real-madrid|         207|    146|       32|      29|
|juventus-turin|         204|    158|       25|      21|
|celtic-glasgow|         198|    147|       28|      23|
+--------------+------------+-------+---------+--------+
only showing top 5 rows

